# SME0110 - Programação Matemática

## Trabalho de Otimizazação Inteira

### Grupo 9:

- Ana Vitória Freitas - 1370196
- Eduardo Rodrigues Amaral - 11735021
- Fernando Henrique Paes Generich - 11795342
- Guilherme Rios - 11222839
- Pedro Augusto Ribeiro Gomes - 11819125


## Instalação das Bibiliotecas Necessárias

In [ ]:
!pip install --quiet gwpy gurobipy >> /dev/null

In [ ]:
%%capture

!pip install condacolab -q

import condacolab

condacolab.install()

In [ ]:
%%capture
!conda install pyscipopt

In [ ]:
!ln -s /usr/local/lib/libscip.so.8.1 /usr/local/lib/libscip.so.8.0

## Importação das Bibliotecas Necessárias

In [ ]:
import os
import json
import datetime
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

import pyscipopt as scip

## Download dos Dados das Instâncias fornecidas pela Docente e da Instância de Dados do Toy Problem

In [ ]:
!wget https://github.com/rolimans/sme0110-t1/raw/main/data/instances.zip -O instances.zip -q
!wget https://github.com/rolimans/sme0110-t1/raw/main/data/toyProblemInstance.txt -O toyProblemInstance.txt -q
!rm -rf instances/
!mkdir instances/
!mv instances.zip instances/
!unzip -qq instances/instances.zip -d instances/
!rm instances/instances.zip

## Configurando Licensa Gurobi

In [ ]:
# INSIRA SUA LICENÇA AQUI
params = {
  "WLSACCESSID": 'INSERT_HERE',
  "WLSSECRET": 'INSERT_HERE',
  "LICENSEID": INSERT_HERE,
}
gurobiEnv = gp.Env(params=params)

## Lendo os Dados das Instâncias fornecidas pela Docente

### Função Auxiliar para Leitura de Arquivos de Instância

In [ ]:
# Extraindo dados dos arquivos de instância
def extractDataFromInstanceFile(filepath):
    f = open(filepath, 'r')
    instanceSize = f.readline()

    # n -> locais onde facilidades podem ser instaladas
    # m -> clientes que devem ser atendidos
    n, m = map(int, instanceSize.split(' '))

    # capacidades e preço fixo de cada facilidade lida do arquivo de instâncias
    capacities = []
    fixedCosts = []
    for _ in range(n):
        line = f.readline()

        capacity, fixedCost = list(map(int, line.strip().split()))

        capacities.append(capacity)
        fixedCosts.append(fixedCost)

    # demanda de cada cliente j
    demands = []
    # custo de transporte da facilidade i para o cliente j
    transportCosts = [[0 for _ in range(m)] for _ in range(n)]

    for j in range(m):
        line = f.readline()
        values = list(map(int, line.strip().split()))

        demands.append(values[0])

        for i, cost in enumerate(values[1:]):
            transportCosts[i][j] = cost

    f.close()

    return {
        'dimensions': (n, m),
        'capacities': capacities,
        'fixedCosts': fixedCosts,
        'demands': demands,
        'transportCosts': transportCosts
    }

### Lendo as instâncias e salvando os dados no dicionário `instancesData`

In [ ]:
INSTANCES_DIR = './instances'

instancesData = {}
for fname in os.listdir(INSTANCES_DIR):
    fpath = f'{INSTANCES_DIR}/{fname}'
    print(f'Read Instance File: {fpath}')
    newData = extractDataFromInstanceFile(fpath)

    instancesData[os.path.splitext(fname)[0]] = newData

Read Instance File: ./instances/Adaptada-wlp04.txt
Read Instance File: ./instances/Adaptada-wlp05.txt
Read Instance File: ./instances/Adaptada-wlp01.txt
Read Instance File: ./instances/Adaptada-wlp03.txt
Read Instance File: ./instances/Adaptada-wlp02.txt


## Tarefa 1

Escreva o modelo de localização de facilidades que minimiza os custos em linguagem de modelagem.

### Modelo de Localização de Facilidades para Minimização de Custos - Gurobi

In [ ]:
# n ->  número de locais onde facilidades podem ser instaladas
# m -> número de clientes que devem ser atendidos
# capacities -> capacidade de cada facilidade
# demands -> demanda de cada cliente
# fixedCosts -> custo fixo de cada facilidade
# transportCosts -> custo de transporte de cada facilidade para cada cliente

def getMinCostModelGurobi(data):
    n, m = data['dimensions']
    capacities = data['capacities']
    demands = data['demands']
    fixedCosts = data['fixedCosts']
    transportCosts = data['transportCosts']

    model = gp.Model(env=gurobiEnv)

    # Definindo variáveis para o modelo
    # Facilidades abertas
    open = model.addVars(n,
                         vtype=GRB.BINARY,
                         obj=fixedCosts,
                         name="open")

    # Fração da demanda do cliente j atendida pela facilidade i
    x = model.addVars(n,
                      m,
                      vtype=GRB.CONTINUOUS,
                      lb=0,
                      ub=1,
                      name='coveredClients')

    # Definindo as restrições do problema
    # Garantindo que todos os clientes tenham sua demanda atendida (2)
    model.addConstrs((gp.quicksum(x[(i,j)] for i in range(n)) == 1 for j in range(m)), name='clientDemand')

    # Garantindo que a demanda total atendida pelas facilidades abertas seja menor ou igual à capacidade dessas facilidades (3)
    for i in range(n):
        model.addConstr(gp.quicksum(demands[j] * x[i,j] for j in range(m)) <= capacities[i] * open[i], name=f'capacityConstr_{i}')

    # Definindo função objetivo (1)
    model.setObjective(
        (gp.quicksum(fixedCosts[i] * open[i] for i in range(n))
        + gp.quicksum(transportCosts[i][j] * x[i,j] for i in range(n) for j in range(m)) ), GRB.MINIMIZE)

    return model

### Modelo de Localização de Facilidades para Minimização de Custos - SCIP

In [ ]:
# n ->  número de locais onde facilidades podem ser instaladas
# m -> número de clientes que devem ser atendidos
# capacities -> capacidade de cada facilidade
# demands -> demanda de cada cliente
# fixedCosts -> custo fixo de cada facilidade
# transportCosts -> custo de transporte de cada facilidade para cada cliente

def getMinCostModelSCIP(data):
    n, m = data['dimensions']
    capacities = data['capacities']
    demands = data['demands']
    fixedCosts = data['fixedCosts']
    transportCosts = data['transportCosts']

    model = scip.Model()

    # Definindo variáveis para o modelo
    # Facilidades abertas
    open = {}
    for i in range(n):
        open[i] = model.addVar(vtype='BINARY', obj=fixedCosts[i], name=f"open_{i}")

    # Fração da demanda do cliente j atendida pela facilidade i
    x = {}
    for i in range(n):
        for j in range(m):
            x[i, j] = model.addVar(vtype='CONTINUOUS', lb=0, ub=1, name=f'coveredClients_{i}_{j}')

    # Definindo as restrições do problema
    # Garantindo que todos os clientes tenham sua demanda atendida (2)
    for j in range(m):
        model.addCons(scip.quicksum(x[(i, j)] for i in range(n)) == 1, name=f'clientDemand_{j}')

    # Garantindo que a demanda total atendida pelas facilidades abertas seja menor ou igual à capacidade dessas facilidades (3)
    for i in range(n):
        model.addCons(scip.quicksum(demands[j] * x[i, j] for j in range(m)) <= capacities[i] * open[i], name=f'capacityConstr_{i}')

    # Definindo função objetivo (1)
    model.setObjective(
        scip.quicksum(fixedCosts[i] * open[i] for i in range(n))
        + scip.quicksum(transportCosts[i][j] * x[i, j] for i in range(n) for j in range(m)), "minimize")

    model.data = {
        'open': open,
        'x': x
    }

    return model

## Tarefa 2

Sabemos que as restrições (3) garantem que as variáveis $y_i$ assumam valores 0 ou 1 em todas as soluções factı́veis. No entanto, quando é resolvida a relaxação linear do problema, estas variáveis podem assumir valores reais. O limitante dual do problema poderia ser melhorado se novas restrições fossem adicionadas ao problema, por exemplo:

$$
x_{ij} \leq y_i
$$

Explique brevemente por que essas restrições poderiam trazer melhorias. Para as instâncias disponibilizadas resolva o problema linearmente relaxado considerando o modelo (1) - (5). Em seguida, resolva novamente as instâncias adicionando as restrições (11) e reescrevendo as restrições (3) como:

$$
\sum_{j=1}^{m} d_jx_{ij} \leq Cap_i \qquad  i = 1, ..., n.
$$

### Modelo de Localização de Facilidades para Minimização de Custos Relaxado e Customizado - Gurobi

In [ ]:
# n ->  número de locais onde facilidades podem ser instaladas
# m -> número de clientes que devem ser atendidos
# capacities -> capacidade de cada facilidade
# demands -> demanda de cada cliente
# fixedCosts -> custo fixo de cada facilidade
# transportCosts -> custo de transporte de cada facilidade para cada cliente

def getMinCostModelRelaxedGurobi(data, extraRestrictions):
    n, m = data['dimensions']
    capacities = data['capacities']
    demands = data['demands']
    fixedCosts = data['fixedCosts']
    transportCosts = data['transportCosts']

    model = gp.Model(env=gurobiEnv)

    # Definindo variáveis para o modelo
    # Facilidades abertas (variável relaxada)
    open = model.addVars(n,
                         vtype=GRB.CONTINUOUS,
                         lb=0,
                         ub=1,
                         obj=fixedCosts,
                         name="open")

    # Fração da demanda do cliente j atendida pela facilidade i
    x = model.addVars(n,
                      m,
                      vtype=GRB.CONTINUOUS,
                      lb=0,
                      ub=1,
                      name='coveredClients')

    # Definindo as restrições do problema
    # Garantindo que todos os clientes tenham sua demanda atendida (2)
    model.addConstrs((gp.quicksum(x[(i,j)] for i in range(n)) == 1 for j in range(m)), name='clientDemand')

    # Garantindo que a demanda total atendida pelas facilidades abertas seja menor ou igual à capacidade dessas facilidades (3)
    if not extraRestrictions:
        for i in range(n):
            model.addConstr(gp.quicksum(demands[j] * x[i,j] for j in range(m)) <= capacities[i] * open[i], name=f'capacityConstr_{i}')
    else:
        # Restrição 3 reescrita
        for i in range(n):
            model.addConstr(gp.quicksum(demands[j] * x[i,j] for j in range(m)) <= capacities[i], name=f'customCapacityConstr_{i}')

    if extraRestrictions:
        # Adicionando a restrição sugerida (11)
        for i in range(n):
            for j in range(m):
                model.addConstr(x[i,j] <= open[i], name=f'newConstr_{i}_{j}')

    # Definindo função objetivo (1)
    model.setObjective(
        (gp.quicksum(fixedCosts[i] * open[i] for i in range(n))
        + gp.quicksum(transportCosts[i][j] * x[i,j] for i in range(n) for j in range(m)) ), GRB.MINIMIZE)

    return model

### Modelo de Localização de Facilidades para Minimização de Custos Relaxado e Customizado - SCIP

In [ ]:
# n ->  número de locais onde facilidades podem ser instaladas
# m -> número de clientes que devem ser atendidos
# capacities -> capacidade de cada facilidade
# demands -> demanda de cada cliente
# fixedCosts -> custo fixo de cada facilidade
# transportCosts -> custo de transporte de cada facilidade para cada cliente

def getMinCostModelRelaxedSCIP(data, extraRestrictions):
    n, m = data['dimensions']
    capacities = data['capacities']
    demands = data['demands']
    fixedCosts = data['fixedCosts']
    transportCosts = data['transportCosts']

    model = scip.Model()

    # Definindo variáveis para o modelo
    # Facilidades abertas (variável relaxada)
    open = {}
    for i in range(n):
        open[i] = model.addVar(vtype='CONTINUOUS', lb=0, ub=1, obj=fixedCosts[i], name=f"open_{i}")

    # Fração da demanda do cliente j atendida pela facilidade i
    x = {}
    for i in range(n):
        for j in range(m):
            x[i, j] = model.addVar(vtype='CONTINUOUS', lb=0, ub=1, name=f'coveredClients_{i}_{j}')

    # Definindo as restrições do problema
    # Garantindo que todos os clientes tenham sua demanda atendida (2)
    for j in range(m):
        model.addCons(scip.quicksum(x[(i, j)] for i in range(n)) == 1, name=f'clientDemand_{j}')

    # Garantindo que a demanda total atendida pelas facilidades abertas seja menor ou igual à capacidade dessas facilidades (3)
    if not extraRestrictions:
      for i in range(n):
          model.addCons(scip.quicksum(demands[j] * x[i, j] for j in range(m)) <= capacities[i] * open[i], name=f'capacityConstr_{i}')
    else:
      # Restrição 3 reescrita
      for i in range(n):
          model.addCons(scip.quicksum(demands[j] * x[i, j] for j in range(m)) <= capacities[i], name=f'customCapacityConstr_{i}')

    if extraRestrictions:
        # Adicionando a restrição sugerida (11)
        for i in range(n):
            for j in range(m):
                model.addCons(x[i,j] <= open[i], name=f'newConstr_{i}_{j}')

    # Definindo função objetivo (1)
    model.setObjective(
        scip.quicksum(fixedCosts[i] * open[i] for i in range(n))
        + scip.quicksum(transportCosts[i][j] * x[i, j] for i in range(n) for j in range(m)), "minimize")

    model.data = {
        'open': open,
        'x': x
    }

    return model

### Resolvendo as Instâncias

Primeiramente, iremos utilizar os dados, retirados das instâncias anteriormente, presentes na variável `instancesData`. Em seguida, iremos resolver o problema relaxado:

In [ ]:
# Resolvendo o problema relaxado com Gurobi

RESULTS_TASK02_1_DIR = './results/resultsTask02/relaxed/gurobi'


if not os.path.exists(RESULTS_TASK02_1_DIR):
	os.makedirs(RESULTS_TASK02_1_DIR)

for instanceName, instanceData in instancesData.items():

    print('-'*100)

    model = getMinCostModelRelaxedGurobi(instanceData, extraRestrictions=False)

    model.setParam('OutputFlag', 0)

    # model.setParam('TimeLimit', 10)

    filepath = f'{RESULTS_TASK02_1_DIR}/resultFor_{instanceName}.lp'
    jsonpath = f'{RESULTS_TASK02_1_DIR}/resultFor_{instanceName}.json'

    if os.path.exists(filepath):
        os.remove(filepath)

    if os.path.exists(jsonpath):
        os.remove(jsonpath)


    print(f'Running Gurobi Relaxed Model on instance: {instanceName}')
    print(f'Writing Result to {filepath}...')

    model.optimize()

    print(f'Model Finished with Status: {model.status}')

    model.write(filepath)  # Salvando o problema em um arquivo .lp

    # Coletando informações após otimização
    dualBound = model.ObjBound  # Limitande Dual
    primalBound = model.objVal  # Limitante Primal
    reachedTimeLimit = model.status == GRB.TIME_LIMIT  # Estourou o tempo limite
    gap = model.MIPGap * 100 if model.getAttr('IsMIP') else None  # Gap (%), se modelo MIP
    executionTime = model.Runtime  # Tempo de Execução
    isOptimal = model.status == GRB.OPTIMAL  # É solução óptima

    results = {
        "instance": instanceName,
        "dualBound": dualBound,
        "primalBound": primalBound,
        "reachedTimeLimit": reachedTimeLimit,
        "gap": gap,
        "executionTime": executionTime,
        "isOptimal": isOptimal
    }

    print(results)

    # Salvar resultados em arquivo JSON
    with open(jsonpath, 'w') as jsonFile:
        json.dump(results, jsonFile)

    print('-'*100)

----------------------------------------------------------------------------------------------------
Running Gurobi Relaxed Model on instance: Adaptada-wlp01
Writing Result to ./results/resultsTask02/relaxed/gurobi/resultFor_Adaptada-wlp01.lp...
Model Finished with Status: 2
{'instance': 'Adaptada-wlp01', 'dualBound': 69005.35745604144, 'primalBound': 69005.35745604144, 'reachedTimeLimit': False, 'gap': None, 'executionTime': 0.875817060470581, 'isOptimal': True}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Running Gurobi Relaxed Model on instance: Adaptada-wlp04
Writing Result to ./results/resultsTask02/relaxed/gurobi/resultFor_Adaptada-wlp04.lp...
Model Finished with Status: 2
{'instance': 'Adaptada-wlp04', 'dualBound': 135069.02353311464, 'primalBound': 135069.02353311464, 'reachedTimeLimit': False, 'gap': None, 'executionTime':

In [ ]:
# Resolvendo o problema relaxado com SCIP

RESULTS_TASK02_2_DIR = './results/resultsTask02/relaxed/SCIP'


if not os.path.exists(RESULTS_TASK02_2_DIR):
	os.makedirs(RESULTS_TASK02_2_DIR)

for instanceName, instanceData in instancesData.items():

    print('-'*100)

    model = getMinCostModelRelaxedSCIP(instanceData, extraRestrictions=False)

    model.hideOutput()

    # model.setRealParam('limits/time', 10)

    filepath = f'{RESULTS_TASK02_2_DIR}/resultFor_{instanceName}.lp'
    jsonpath = f'{RESULTS_TASK02_2_DIR}/resultFor_{instanceName}.json'

    if os.path.exists(filepath):
        os.remove(filepath)

    if os.path.exists(jsonpath):
        os.remove(jsonpath)


    print(f'Running SCIP Relaxed Model on instance: {instanceName}')
    print(f'Writing Result to {filepath}...')

    model.optimize()

    print(f'Model Finished with Status: {model.getStatus()}')

    model.writeProblem(filepath)  # Salvando o problema em um arquivo .lp

    # Coletando informações após otimização
    dualBound = model.getDualbound()  # Limitande Dual
    primalBound = model.getPrimalbound()  # Limitante Primal
    reachedTimeLimit = model.getStatus() == "timelimit"  # Estourou o tempo limite
    gap = model.getGap() * 100  # Gap (%), se modelo MIP
    executionTime = model.getSolvingTime()  # Tempo de Execução
    isOptimal = model.getStatus() == "optimal"  # É solução óptima

    results = {
        "instance": instanceName,
        "dualBound": dualBound,
        "primalBound": primalBound,
        "reachedTimeLimit": reachedTimeLimit,
        "gap": gap,
        "executionTime": executionTime,
        "isOptimal": isOptimal
    }

    print(results)

    # Salvar resultados em arquivo JSON
    with open(jsonpath, 'w') as jsonFile:
        json.dump(results, jsonFile)

    print('-'*100)

----------------------------------------------------------------------------------------------------
Running SCIP Relaxed Model on instance: Adaptada-wlp01
Writing Result to ./results/resultsTask02/relaxed/SCIP/resultFor_Adaptada-wlp01.lp...
Model Finished with Status: optimal
wrote problem to file /content/results/resultsTask02/relaxed/SCIP/resultFor_Adaptada-wlp01.lp
{'instance': 'Adaptada-wlp01', 'dualBound': 69005.35745604156, 'primalBound': 69005.35745604156, 'reachedTimeLimit': False, 'gap': 0.0, 'executionTime': 18.271101, 'isOptimal': True}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Running SCIP Relaxed Model on instance: Adaptada-wlp04
Writing Result to ./results/resultsTask02/relaxed/SCIP/resultFor_Adaptada-wlp04.lp...
Model Finished with Status: optimal
wrote problem to file /content/results/resultsTask02/relaxed/SCIP/

A seguir, iremos adicionar a restrição (11) e vamos alterar a restrição (3) como:
$$
\sum_{j=1}^{m} d_jx_{ij} \leq Cap_i \qquad  i = 1, ..., n.
$$

Então resolveremos o problema com o novo modelo.

In [ ]:
# Resolvendo o problema relaxado e customizado com Gurobi

RESULTS_TASK02_3_DIR = './results/resultsTask02/customRelaxed/gurobi'


if not os.path.exists(RESULTS_TASK02_3_DIR):
	os.makedirs(RESULTS_TASK02_3_DIR)

for instanceName, instanceData in instancesData.items():

    print('-'*100)

    model = getMinCostModelRelaxedGurobi(instanceData, extraRestrictions=True)

    model.setParam('OutputFlag', 0)
    # model.setParam('TimeLimit', 10)

    filepath = f'{RESULTS_TASK02_3_DIR}/resultFor_{instanceName}.lp'
    jsonpath = f'{RESULTS_TASK02_3_DIR}/resultFor_{instanceName}.json'

    if os.path.exists(filepath):
        os.remove(filepath)

    if os.path.exists(jsonpath):
        os.remove(jsonpath)


    print(f'Running Gurobi Custom Relaxed Model on instance: {instanceName}')
    print(f'Writing Result to {filepath}...')

    model.optimize()

    print(f'Model Finished with Status: {model.status}')

    model.write(filepath)  # Salvando o problema em um arquivo .lp

    # Coletando informações após otimização
    dualBound = model.ObjBound  # Limitande Dual
    primalBound = model.objVal  # Limitante Primal
    reachedTimeLimit = model.status == GRB.TIME_LIMIT  # Estourou o tempo limite
    gap = model.MIPGap * 100 if model.getAttr('IsMIP') else None  # Gap (%), se modelo MIP
    executionTime = model.Runtime  # Tempo de Execução
    isOptimal = model.status == GRB.OPTIMAL  # É solução óptima

    results = {
        "instance": instanceName,
        "dualBound": dualBound,
        "primalBound": primalBound,
        "reachedTimeLimit": reachedTimeLimit,
        "gap": gap,
        "executionTime": executionTime,
        "isOptimal": isOptimal
    }

    print(results)

    # Salvar resultados em arquivo JSON
    with open(jsonpath, 'w') as jsonFile:
        json.dump(results, jsonFile)

    print('-'*100)

----------------------------------------------------------------------------------------------------
Running Gurobi Custom Relaxed Model on instance: Adaptada-wlp01
Writing Result to ./results/resultsTask02/customRelaxed/gurobi/resultFor_Adaptada-wlp01.lp...
Model Finished with Status: 2
{'instance': 'Adaptada-wlp01', 'dualBound': 12808.630101981842, 'primalBound': 12808.630101981842, 'reachedTimeLimit': False, 'gap': None, 'executionTime': 74.20181393623352, 'isOptimal': True}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Running Gurobi Custom Relaxed Model on instance: Adaptada-wlp04
Writing Result to ./results/resultsTask02/customRelaxed/gurobi/resultFor_Adaptada-wlp04.lp...
Model Finished with Status: 2
{'instance': 'Adaptada-wlp04', 'dualBound': 20797.084755175973, 'primalBound': 20797.084755175973, 'reachedTimeLimit': False, '

In [ ]:
# Resolvendo o problema relaxado e customizado com SCIP

RESULTS_TASK02_4_DIR = './results/resultsTask02/customRelaxed/SCIP'


if not os.path.exists(RESULTS_TASK02_4_DIR):
	os.makedirs(RESULTS_TASK02_4_DIR)

for instanceName, instanceData in instancesData.items():

    print('-'*100)

    model = getMinCostModelRelaxedSCIP(instanceData, extraRestrictions=True)

    model.hideOutput()

    # model.setRealParam('limits/time', 10)

    filepath = f'{RESULTS_TASK02_4_DIR}/resultFor_{instanceName}.lp'
    jsonpath = f'{RESULTS_TASK02_4_DIR}/resultFor_{instanceName}.json'

    if os.path.exists(filepath):
        os.remove(filepath)

    if os.path.exists(jsonpath):
        os.remove(jsonpath)


    print(f'Running SCIP Custom Relaxed Model on instance: {instanceName}')
    print(f'Writing Result to {filepath}...')

    model.optimize()

    print(f'Model Finished with Status: {model.getStatus()}')

    model.writeProblem(filepath)  # Salvando o problema em um arquivo .lp

    # Coletando informações após otimização
    dualBound = model.getDualbound()  # Limitande Dual
    primalBound = model.getPrimalbound()  # Limitante Primal
    reachedTimeLimit = model.getStatus() == "timelimit"  # Estourou o tempo limite
    gap = model.getGap() * 100  # Gap (%), se modelo MIP
    executionTime = model.getSolvingTime()  # Tempo de Execução
    isOptimal = model.getStatus() == "optimal"  # É solução óptima

    results = {
        "instance": instanceName,
        "dualBound": dualBound,
        "primalBound": primalBound,
        "reachedTimeLimit": reachedTimeLimit,
        "gap": gap,
        "executionTime": executionTime,
        "isOptimal": isOptimal
    }

    print(results)

    # Salvar resultados em arquivo JSON
    with open(jsonpath, 'w') as jsonFile:
        json.dump(results, jsonFile)

    print('-'*100)

----------------------------------------------------------------------------------------------------
Running SCIP Custom Relaxed Model on instance: Adaptada-wlp01
Writing Result to ./results/resultsTask02/customRelaxed/SCIP/resultFor_Adaptada-wlp01.lp...
Model Finished with Status: timelimit
{'instance': 'Adaptada-wlp01', 'dualBound': 12808.63010198189, 'primalBound': 12808.63010198189, 'reachedTimeLimit': False, 'gap': 0.0, 'executionTime': 610.107975, 'isOptimal': True}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Running SCIP Custom Relaxed Model on instance: Adaptada-wlp02
Writing Result to ./results/resultsTask02/customRelaxed/SCIP/resultFor_Adaptada-wlp02.lp...
Model Finished with Status: timelimit
{'instance': 'Adaptada-wlp02', 'dualBound': 14029.79648947291, 'primalBound': 14029.79648947291, 'reachedTimeLimit': False, 'gap'

<font color="red" ><i>Obs</i>:</font> Apenas as instâncias 1, 2 e 3 foram executadas no modelo relaxado e customizado que utiliza o *solver* não-comercial **SCIP**, pois, as outras instâncias gastaram mais tempo do que o factível para o trabalho (4h + por instância).

## Tarefa 3

Resolva as instâncias disponibilizadas no site da disciplina utilizando o *software* não-comercial de otimização SCIP, com tempo limite de **5 minutos** para cada instância.

In [ ]:
# Resolvendo o problema com tempo limite de 5 minutos
RESULTS_TASK03_DIR = './results/resultsTask03'

if not os.path.exists(RESULTS_TASK03_DIR):
	os.makedirs(RESULTS_TASK03_DIR)

for instanceName, instanceData in instancesData.items():

    print('-'*100)

    model = getMinCostModelSCIP(instanceData)

    # Configurando tempo limite
    model.hideOutput()
    model.setRealParam('limits/time', 5 * 60)
    # model.setRealParam('limits/time', 10)

    filepath = f'{RESULTS_TASK03_DIR}/resultFor_{instanceName}.lp'
    jsonpath = f'{RESULTS_TASK03_DIR}/resultFor_{instanceName}.json'

    if os.path.exists(filepath):
        os.remove(filepath)

    if os.path.exists(jsonpath):
        os.remove(jsonpath)

    print(f'Running SCIP Model on instance: {instanceName}')
    print(f'Writing Result to {filepath}...')

    model.optimize()

    print(f'Model Finished with Status: {model.getStatus()}')

    model.writeProblem(filepath)  # Salvando o problema em um arquivo .lp

    # Coletando informações após otimização
    dualBound = model.getDualbound()  # Limitande Dual
    primalBound = model.getPrimalbound()  # Limitante Primal
    reachedTimeLimit = model.getStatus() == "timelimit"  # Estourou o tempo limite
    gap = model.getGap() * 100  # Gap (%), se modelo MIP
    executionTime = model.getSolvingTime()  # Tempo de Execução
    isOptimal = model.getStatus() == "optimal"  # É solução óptima

    results = {
        "instance": instanceName,
        "dualBound": dualBound,
        "primalBound": primalBound,
        "reachedTimeLimit": reachedTimeLimit,
        "gap": gap,
        "executionTime": executionTime,
        "isOptimal": isOptimal
    }

    print(results)

    # Salvar resultados em arquivo JSON
    with open(jsonpath, 'w') as jsonFile:
        json.dump(results, jsonFile)

    print('-'*100)

----------------------------------------------------------------------------------------------------
Running SCIP Model on instance: Adaptada-wlp01
Writing Result to ./results/resultsTask03/resultFor_Adaptada-wlp01.lp...
Model Finished with Status: timelimit
wrote problem to file /content/results/resultsTask03/resultFor_Adaptada-wlp01.lp
{'instance': 'Adaptada-wlp01', 'dualBound': 69042.96272321424, 'primalBound': 69335.52771322845, 'reachedTimeLimit': True, 'gap': 0.42374338886220386, 'executionTime': 300.156489, 'isOptimal': False}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Running SCIP Model on instance: Adaptada-wlp04
Writing Result to ./results/resultsTask03/resultFor_Adaptada-wlp04.lp...
Model Finished with Status: timelimit
wrote problem to file /content/results/resultsTask03/resultFor_Adaptada-wlp04.lp
{'instance': 'Adapt

## Tarefa 4

Resolver as instâncias disponibilizadas no site da disciplina utilizando o *software* comercial de otimização GUROBI com tempo limite de **5 minutos** para cada instância.

In [ ]:
# Resolvendo o problema com tempo limite de 5 minutos
RESULTS_TASK04_DIR = './results/resultsTask04'

if not os.path.exists(RESULTS_TASK04_DIR):
	os.makedirs(RESULTS_TASK04_DIR)

for instanceName, instanceData in instancesData.items():

    print('-'*100)

    model = getMinCostModelGurobi(instanceData)

    # Configurando tempo limite
    model.setParam('OutputFlag', 0)
    model.setParam('TimeLimit', 5 * 60)
    # model.setParam('TimeLimit', 10)

    filepath = f'{RESULTS_TASK04_DIR}/resultFor_{instanceName}.lp'
    jsonpath = f'{RESULTS_TASK04_DIR}/resultFor_{instanceName}.json'

    if os.path.exists(filepath):
        os.remove(filepath)

    if os.path.exists(jsonpath):
        os.remove(jsonpath)

    print(f'Running Gurobi Model on instance: {instanceName}')
    print(f'Writing Result to {filepath}...')

    model.optimize()

    print(f'Model Finished with Status: {model.status}')

    model.write(filepath)  # Salvando o problema em um arquivo .lp

    # Coletando informações após otimização
    dualBound = model.ObjBound  # Limitande Dual
    primalBound = model.objVal  # Limitante Primal
    reachedTimeLimit = model.status == GRB.TIME_LIMIT  # Estourou o tempo limite
    gap = model.MIPGap * 100 if model.getAttr('IsMIP') else None  # Gap (%), se modelo MIP
    executionTime = model.Runtime  # Tempo de Execução
    isOptimal = model.status == GRB.OPTIMAL  # É solução óptima

    results = {
        "instance": instanceName,
        "dualBound": dualBound,
        "primalBound": primalBound,
        "reachedTimeLimit": reachedTimeLimit,
        "gap": gap,
        "executionTime": executionTime,
        "isOptimal": isOptimal
    }

    print(results)

    # Salvar resultados em arquivo JSON
    with open(jsonpath, 'w') as jsonFile:
        json.dump(results, jsonFile)

    print('-'*100)

----------------------------------------------------------------------------------------------------
Running Gurobi Model on instance: Adaptada-wlp01
Writing Result to ./results/resultsTask04/resultFor_Adaptada-wlp01.lp...
Model Finished with Status: 2
{'instance': 'Adaptada-wlp01', 'dualBound': 69049.63751309768, 'primalBound': 69055.24702235329, 'reachedTimeLimit': False, 'gap': 0.008123219447452402, 'executionTime': 14.10982608795166, 'isOptimal': True}
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Running Gurobi Model on instance: Adaptada-wlp04
Writing Result to ./results/resultsTask04/resultFor_Adaptada-wlp04.lp...
Model Finished with Status: 9
{'instance': 'Adaptada-wlp04', 'dualBound': 135108.44077295566, 'primalBound': 135143.45585477047, 'reachedTimeLimit': True, 'gap': 0.025909565204872916, 'executionTime': 300.0477058887

## Tarefa 5

Pesquise uma aplicação do problema de localização de facilidades. Descreva a aplicação, explique quais são os parâmetros, suas variáveis, a função objetivo e suas restrições. Em resumo, deixe clara sua aplicação e justifique sua utilidade. Obs. Não podem ser utilizados os exemplos apresentados em aula.

### Aplicação do Problema de Facilidades


#### Ideia Geral do Problema

Imaginemos uma situação em que uma empresa de logística precisa decidir onde construir centros de distribuição para minimizar o custo operacional e maximizar a cobertura de clientes.

Foram identificados vários locais candidatos para os centros de distribuição, mas devem ser tomadas decisões com relação a quantos armazéns abrir e em quais localidades.

Abrir muitos armazéns seria vantajoso, pois aumentaria a cobertura de clientes. No entanto, a abertura de um centro de distribuição tem um custo fixo associado e o transporte de mercadorias até o cliente também tem um custo variável de acordo com o centro de distribuição e o cliente.

Neste exemplo, nosso objetivo é encontrar o equilíbrio ideal entre a cobertura oferecida pelos centros de distribuições abertos e os custos de construção e operação dos mesmos.

Como temos mais de um objetivo neste problema (maximizar cobertura e minimizar custos) utilizaremos a estratégia de Agregação Ponderada, na função objetivo, de modo a atribuir diferentes pesos a cada objetivo. Caso seja desejado priorizar um objetivo a outro, basta atribuir-se um peso maior ao objetivo principal.


####  Formulação do Problema

### Conjuntos e Índices

$i \in I$: Índice e conjunto dos centros de distribuição (ou facilidades).

$j \in J$: Índice e conjunto dos clientes.

$m$: Número de clientes

### Parâmetros

$f_{i} \in \mathbb{R}^+$: Custo fixo associado a construção do centro de distribuição $i \in I$.

$d_{j} \in \mathbb{R}^+$: Demanda máxima do cliente $j \in J$.

$Cap_i \in \mathbb{R}^+$ : é a capacidade do centro de distribuição $i \in I$.

$c_{i,j} \in \mathbb{R}^+$: Custo de transporte entre o centro de distribuição candidato $i \in I$ e a localidade do cliente $j \in J$.

$w_{custo} \in \mathbb{R}^+$: Peso do objetivo de minimizar os custos totais.

$w_{cobertura} \in \mathbb{R}^+$: Peso do objetivo de maximizar cobertura total.

### Variáveis de decisão

$select_{i} \in \{0, 1 \}$: Variável é 1 se o centro de distribuição é construído na localidade candidata $i \in I$; e 0 caso contrário.

$0 \leq assign_{i,j} \leq 1$: Variável contínua, não negativa que determina a fração de cobertura do cliente $j \in J$ pelo centro de distribuição $i \in I$.

### Função Objetivo

- **Custos Totais e Cobertura Total**. Queremos minimizar os custos totais de abrir e operar os centros de distribuição e ao mesmo tempo maximizar a cobertura de clientes. Para isso utilizaremos uma função objetivo agregada ponderada.

 - Para os custos totais, temos a soma do custo fixo de abrir centros de distribuição e o custo relacionado ao transporte entre centros de distribuição e clientes dividida pela soma de todos custos fixos e variáveis (normalizada).

 - Para a cobertura total temos a soma das coberturas de cada cliente por cada centro de distribuição dividida pelo número de clientes (normalizada). Como o objetivo é aumentar a cobertura e a função objetivo é de minimização, utiliza-se o oposto do valor encontrado.

 - Por fim calcula-se a soma ponderada dos objetivos com seus respectivos pesos dividida pela soma total dos pesos.

\begin{equation}
\text{Min} \quad Z =
(\frac{ \sum_{i \in I} f_{i} \cdot select_{i} + \sum_{i \in I} \sum_{j \in J} c_{i,j} \cdot assign_{i,j} }{ \sum_{i \in I} f_{i} + \sum_{i \in I} \sum_{j \in J} c_{i,j} }) \cdot w_{custo}
+
( - {\frac{\sum_{i \in I} \sum_{j \in J} assign_{i,j}}{m}) \cdot w_{cobertura}}
\tag{0}
\end{equation}

### Restrições

- **Demanda**. Para cada cliente  $i \in I$ assegura-se que sua demanda máxima é respeitada. Sendo assim, a soma da fração recebida de cada facilidade para cada cliente deve ser menor ou igual a 1:

\begin{equation}
\sum_{i \in I} assign_{i,j} \leq 1 \quad \forall j \in J
\tag{1}
\end{equation}

- **Capacidade**. É preciso garantir que a facilidade $i \in I$, lide no máximo com sua capacidade máxima, caso ela tenha sido construída.

\begin{equation}
\sum_{j \in J} d_j \cdot assign_{i,j} \leq Cap_i \cdot select_{i} \quad \forall i \in I
\tag{2}
\end{equation}

## Tarefa 6

Elabore um problema exemplo (*toy problem*), descreva-o matematicamente utilizando o modelo e resolva-o utilizando o *solver* não-comercial **SCIP**.

Este exemplo deve ser pequeno em relação a sua dimensão. Ele deve ser utilizado para facilitar a compreensão do problema, do modelo e de suas restrições. Ele deve ser um exemplo para a aplicação escolhida na Tarefa 5.

### Exemplificação do Problema de Facilidades

A campanha do agasalho em São Carlos, possui o seguinte lema: "*Doe agasalhos: aqueça a noite e o coração de alguém*". Para que a campanha seja um sucesso de arrecadações, é preciso pensar nas seguintes estratégias: onde instalar um ponto de coleta na cidade? Quais são as localidades candidatas? Quais os custos de transporte dos pontos de coleta até as instituições que irão receber essas peças de roupa? Qual o cuso fixo de construção de um ponto de coleta? Como aumentar a cobertura da demanda das instituições que irão receber essas peças de roupa?

Para reduzir o conjunto das possíveis localidades, a prefeitura de São Carlos nos mostrou um mapa com alguns locais estratégicos para construção dos pontos de coleta:

![mapa.png](https://github.com/rolimans/sme0110-t1/raw/main/data/map.jpg)

As localidadades candidatas podem ser identificadas pelos pontos verdes no mapa.

Para arrecadar ainda mais agasalhos durante a campanha, a prefeitura pensou em construir pontos de coletas utilizando contâiners metálicos para preservar as peças de roupa da chuva e poeira, além de permitir que as pessoas apenas depositem as roupas, impedindo que alguém sem a chave as retirem do ponto de arrecação. Dessa forma, cada ponto de coleta terá um custo fixo associado na sua construção e manutenção.

Além disso, a prefeitura deseja enviar os agasalhos arrecadados não apenas para as instituições de São Carlos, mas também para as cidades vizinhas como: Água vermelha, Ibaté, Descalvado, Riberão Bonito, etc. Essas instituições também possuem as devidas demandas de agasalho conforme as necessidades da sua comunidade.

Portanto, é preciso minimizar os custos de construir os contâiners metálicos de arrecadação e os custos de transporte das peças de roupas até as instituições dentro e fora de São Carlos e ao mesmo tempo maximizar a cobertura da demanda das instituições participantes pelos pontos de coleta construídos.

### Formalização do Exemplo

### Conjuntos e Índices

$i \in I$: Índice e conjunto das localidades candidatas dos pontos de coleta.

$j \in J$: Índice e conjunto das localidades das instituições participantes.

$m$: Número de instituições participantes

### Parâmetros

$f_{i} \in \mathbb{R}^+$: Custo fixo associado a construção dos pontos de coleta $i \in I$.

$d_{j} \in \mathbb{R}^+$: Demanda máxima da instituição participante $j \in J$.

$Cap_i \in \mathbb{R}^+$ : é a capacidade do ponto de coleta $i \in I$.

$c_{i,j} \in \mathbb{R}^+$: Custo de transporte entre o ponto de coleta candidato $i \in I$ e a localidade da instituição $j \in J$.

$w_{custo} \in \mathbb{R}^+$: Peso do objetivo de minimizar os custos totais.

$w_{cobertura} \in \mathbb{R}^+$: Peso do objetivo de maximizar cobertura total.

### Variáveis de Decisão

$select_{i} \in \{0, 1 \}$: Variável é 1 se o ponto de coleta é construído na localidade candidata $i \in I$; e 0 caso contrário.

$0 \leq assign_{i,j} \leq 1$: Variável contínua, não negativa que determina a fração da demanda de peças de roupa recebida pela instituição $j \in J$ do ponto de coleta $i \in I$.

### Função Objetivo

- **Custos Totais e Cobertura Total**. Queremos minimizar os custos totais de abrir e operar os pontos de coleta e ao mesmo tempo maximizar a cobertura da demanda das instituições participantes. Para isso utilizaremos uma função objetivo agregada ponderada.

 - Para os custos totais, temos a soma do custo fixo de abrir pontos de coleta e o custo relacionado ao transporte entre pontos de coleta e as instituições participantes dividida pela soma de todos custos fixos e variáveis (normalizada).

 - Para a cobertura total temos a soma das coberturas de cada instituição participante por cada ponto de coleta dividida pelo número de instituições participantes (normalizada). Como o objetivo é aumentar a cobertura e a função objetivo é de minimização, utiliza-se o oposto do valor encontrado.

 - Por fim calcula-se a soma ponderada dos objetivos com seus respectivos pesos dividida pela soma total dos pesos.

\begin{equation}
\text{Min} \quad Z =
(\frac{ \sum_{i \in I} f_{i} \cdot select_{i} + \sum_{i \in I} \sum_{j \in J} c_{i,j} \cdot assign_{i,j} }{ \sum_{i \in I} f_{i} + \sum_{i \in I} \sum_{j \in J} c_{i,j} }) \cdot w_{custo}
+
( - {\frac{\sum_{i \in I} \sum_{j \in J} assign_{i,j}}{m}) \cdot w_{cobertura}}
\tag{0}
\end{equation}

### Restrições

- **Demanda**. Para cada instituição participante $j \in J$ assegura-se que sua demanda máxima é respeitada. Sendo assim, a soma da fração recebida de cada ponto de coleta para cada instituição deve ser menor ou igual a 1:

\begin{equation}
\sum_{i \in I} assign_{i,j} \leq 1 \quad \forall j \in J
\tag{1}
\end{equation}

- **Capacidade**. É preciso garantir que o ponto de coleta $i \in I$, lide no máximo com sua capacidade máxima, caso ele tenha sido construída.

\begin{equation}
\sum_{j \in J} d_j \cdot assign_{i,j} \leq Cap_i \cdot select_{i} \quad \forall i \in I
\tag{2}
\end{equation}

### Modelo de Localização de Facilidades para Múltiplos Objetivos - SCIP

In [ ]:
# n ->  número de locais onde pontos de coleta podem ser instalados
# m -> número de instituições participantes que devem ser atendidas
# capacities -> capacidade de cada ponto de coleta
# demands -> demanda de cada instituição participante
# fixedCosts -> custo fixo de cada ponto de coleta
# transportCosts -> custo de transporte de cada ponto de coleta para cada instituição participante

def getToyProblemModelSCIP(data, wCost = 1, wCoverage = 1):
    n, m = data['dimensions']
    capacities = data['capacities']
    demands = data['demands']
    fixedCosts = data['fixedCosts']
    transportCosts = data['transportCosts']

    model = scip.Model()

    # Definindo variáveis para o modelo
    # Pontos de coleta abertos
    select = {}
    for i in range(n):
        select[i] = model.addVar(vtype='BINARY', obj=fixedCosts[i], name=f"select_{i}")

    # Fração da demanda da instituição participante j atendida pelo ponto de coleta i
    assign = {}
    for i in range(n):
        for j in range(m):
            assign[i, j] = model.addVar(vtype='CONTINUOUS', lb=0, ub=1, name=f'coveredClients_{i}_{j}')

    # Definindo as restrições do problema
    # Garantindo que todos as instituições participantes tenham sua demanda máxima respeitada (1)
    for j in range(m):
        model.addCons(scip.quicksum(assign[(i, j)] for i in range(n)) <= 1, name=f'clientDemand_{j}')

    # Garantindo que a demanda total atendida pelos pontos de coleta abertos seja menor ou igual à capacidade desses pontos de coleta (2)
    for i in range(n):
        model.addCons(scip.quicksum(demands[j] * assign[i, j] for j in range(m)) <= capacities[i] * select[i], name=f'capacityConstr_{i}')


    # Definindo função objetivo (0)

    sumOfCosts = (sum(fixedCosts[i] for i in range(n))
        +sum(transportCosts[i][j] for i in range(n) for j in range(m)))

    totalCost = (scip.quicksum(fixedCosts[i] * select[i] for i in range(n))
        + scip.quicksum(transportCosts[i][j] * assign[i, j] for i in range(n) for j in range(m))) / sumOfCosts

    totalCoverage = scip.quicksum(assign[i, j] for i in range(n) for j in range(m)) / m

    model.setObjective(
        totalCost * wCost - totalCoverage * wCoverage,
        "minimize"
    )

    model.data = {
        'select': select,
        'assign': assign

    }

    return model


### Lendo dados da instância do *Toy Problem*

In [ ]:
toyProblemData = extractDataFromInstanceFile('./toyProblemInstance.txt')

toyProblemData

{'dimensions': (5, 4),
 'capacities': [20, 22, 17, 19, 18],
 'fixedCosts': [12000, 15000, 17000, 13000, 16000],
 'demands': [15, 18, 14, 20],
 'transportCosts': [[4000, 2500, 1200, 2200],
  [2000, 2600, 1800, 2600],
  [3000, 3400, 2600, 3100],
  [2500, 3000, 4100, 3700],
  [4500, 4000, 3000, 3200]]}

### Resolvendo Problema com o Modelo Criado Acima

In [ ]:
RESULTS_TASK06_DIR = './results/resultsTask06'

if not os.path.exists(RESULTS_TASK06_DIR):
	os.makedirs(RESULTS_TASK06_DIR)


print('-'*100)

model = getToyProblemModelSCIP(toyProblemData)
model.hideOutput()

filepath = f'{RESULTS_TASK06_DIR}/resultFor_toyProblem.lp'
jsonpath = f'{RESULTS_TASK06_DIR}/resultFor_toyProblem.json'

if os.path.exists(filepath):
    os.remove(filepath)

if os.path.exists(jsonpath):
    os.remove(jsonpath)

print(f'Running SCIP Model on instance: toyProblem')
print(f'Writing Result to {filepath}...')

model.optimize()

print(f'Model Finished with Status: {model.getStatus()}')

model.writeProblem(filepath)  # Salvando o problema em um arquivo .lp

# Coletando informações após otimização
dualBound = model.getDualbound()  # Limitande Dual
primalBound = model.getPrimalbound()  # Limitante Primal
reachedTimeLimit = model.getStatus() == "timelimit"  # Estourou o tempo limite
gap = model.getGap() * 100  # Gap (%), se modelo MIP
executionTime = model.getSolvingTime()  # Tempo de Execução
isOptimal = model.getStatus() == "optimal"  # É solução óptima

results = {
    "instance": 'toyProblem',
    "dualBound": dualBound,
    "primalBound": primalBound,
    "reachedTimeLimit": reachedTimeLimit,
    "gap": gap,
    "executionTime": executionTime,
    "isOptimal": isOptimal
}

print(results)

# Salvar resultados em arquivo JSON
with open(jsonpath, 'w') as jsonFile:
    json.dump(results, jsonFile)

print('-'*100)

----------------------------------------------------------------------------------------------------
Running SCIP Model on instance: toyProblem
Writing Result to ./results/resultsTask06/resultFor_toyProblem.lp...
Model Finished with Status: optimal
wrote problem to file /content/results/resultsTask06/resultFor_toyProblem.lp
{'instance': 'toyProblem', 'dualBound': -0.561868686868687, 'primalBound': -0.561868686868687, 'reachedTimeLimit': False, 'gap': 0.0, 'executionTime': 0.002433, 'isOptimal': True}
----------------------------------------------------------------------------------------------------


## Análise dos Resultados

### Download dos Dados de Resultados

In [ ]:
!wget https://github.com/rolimans/sme0110-t1/raw/main/results.zip -O results.zip -q
!wget https://github.com/rolimans/sme0110-t1/raw/main/results.z01 -O results.z01 -q
!rm -rf results/
!zip -qq -s0 results.zip --out joined.zip
!unzip -qq joined.zip
!rm joined.zip results.z*

### Leitura dos Dados de Resultados

In [ ]:
# Diretório raiz onde estão os arquivos
resultsDir = 'results'

# Função para percorrer os diretórios e ler os arquivos JSON
def readResults(directory):
    dataDict = {}
    for root, dirs, files in os.walk(directory):
        currentDict = dataDict
        for dir_ in os.path.relpath(root, directory).split(os.sep):
            currentDict = currentDict.setdefault(dir_, {})

        for file in files:
            if file.endswith('.json'):
                filepath = os.path.join(root, file)
                with open(filepath, 'r') as json_file:
                    data = json.load(json_file)
                    # Lendo o arquivo JSON e armazenando no dicionário aninhado
                    filename = os.path.splitext(file)[0]
                    currentDict[filename.replace('resultFor_', '')] = data
    return dataDict

# Chamando a função para ler os arquivos JSON e criar o dicionário aninhado de DataFrames
resultsData = readResults(resultsDir)

# pd.set_option('display.float_format', lambda x: '%E' % x)

In [ ]:
scipDf = pd.DataFrame(resultsData['resultsTask03']).transpose().set_index('instance')
gurobiDf = pd.DataFrame(resultsData['resultsTask04']).transpose().set_index('instance')

mergedDf = pd.merge(scipDf, gurobiDf, on='instance', suffixes=('_SCIP', '_Gurobi'))

mergedDf['primalBoundDifference'] = abs(mergedDf['primalBound_SCIP'] - mergedDf['primalBound_Gurobi'])
mergedDf['dualBoundDifference'] = abs(mergedDf['dualBound_SCIP'] - mergedDf['dualBound_Gurobi'])
mergedDf['gapDifference'] = abs(mergedDf['gap_SCIP'] - mergedDf['gap_Gurobi'])
mergedDf['timeDifference'] = abs(mergedDf['executionTime_SCIP'] - mergedDf['executionTime_Gurobi'])

mergedDf['dualBound_SCIP'] = mergedDf['dualBound_SCIP'].map(lambda x : '%E' % x)
mergedDf['dualBoundDifference'] = mergedDf['dualBoundDifference'].map(lambda x : '%E' % x)
mergedDf['primalBound_SCIP'] = mergedDf['primalBound_SCIP'].map(lambda x : '%E' % x)
mergedDf['primalBoundDifference'] = mergedDf['primalBoundDifference'].map(lambda x : '%E' % x)
mergedDf['reachedTimeLimit_SCIP'] = mergedDf['reachedTimeLimit_SCIP'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['gap_SCIP'] = mergedDf['gap_SCIP'].map(lambda x : '%.2f%%' % x)
mergedDf['gapDifference'] = mergedDf['gapDifference'].map(lambda x : '%.2f%%' % x)
mergedDf['isOptimal_SCIP'] = mergedDf['isOptimal_SCIP'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['executionTime_SCIP'] = mergedDf['executionTime_SCIP'].map(lambda x : str(datetime.timedelta(seconds=x)))
mergedDf['dualBound_Gurobi'] = mergedDf['dualBound_Gurobi'].map(lambda x : '%E' % x)
mergedDf['primalBound_Gurobi'] = mergedDf['primalBound_Gurobi'].map(lambda x : '%E' % x)
mergedDf['reachedTimeLimit_Gurobi'] = mergedDf['reachedTimeLimit_Gurobi'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['timeDifference'] = mergedDf['timeDifference'].map(lambda x : str(datetime.timedelta(seconds=x)))
mergedDf['gap_Gurobi'] = mergedDf['gap_Gurobi'].map(lambda x : '%.2f%%' % (0 if x is None else x))
mergedDf['isOptimal_Gurobi'] = mergedDf['isOptimal_Gurobi'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['executionTime_Gurobi'] = mergedDf['executionTime_Gurobi'].map(lambda x : str(datetime.timedelta(seconds=x)))

mergedDf = mergedDf[['dualBound_SCIP','dualBound_Gurobi','dualBoundDifference','primalBound_SCIP','primalBound_Gurobi','primalBoundDifference','reachedTimeLimit_SCIP','reachedTimeLimit_Gurobi','gap_SCIP','gap_Gurobi','gapDifference','executionTime_SCIP','executionTime_Gurobi','timeDifference','isOptimal_SCIP','isOptimal_Gurobi']]

mergedDf

,dualBound_SCIP,dualBound_Gurobi,dualBoundDifference,primalBound_SCIP,primalBound_Gurobi,primalBoundDifference,reachedTimeLimit_SCIP,reachedTimeLimit_Gurobi,gap_SCIP,gap_Gurobi,gapDifference,executionTime_SCIP,executionTime_Gurobi,timeDifference,isOptimal_SCIP,isOptimal_Gurobi
instance,,,,,,,,,,,,,,,,
Adaptada-wlp01,6.904296E+04,6.904964E+04,6.674790E+00,6.933553E+04,6.905525E+04,2.802807E+02,Sim,Não,0.42%,0.01%,0.42%,0:05:00.156489,0:00:14.109826,0:04:46.046663,Não,Sim
Adaptada-wlp04,1.351013E+05,1.351084E+05,7.117161E+00,1.408444E+05,1.351435E+05,5.700961E+03,Sim,Sim,4.25%,0.03%,4.23%,0:05:02.141046,0:05:00.047706,0:00:02.093340,Não,Não
Adaptada-wlp05,-1.000000E+20,1.619544E+05,1.000000E+20,1.000000E+20,1.620220E+05,1.000000E+20,Sim,Sim,10000000000000000000000.00%,0.04%,10000000000000000000000.00%,0:06:30.949473,0:05:00.109095,0:01:30.840378,Não,Não
Adaptada-wlp02,7.587056E+04,7.589386E+04,2.330013E+01,7.593268E+04,7.593268E+04,2.910383E-11,Sim,Sim,0.08%,0.05%,0.03%,0:05:00.053748,0:05:00.028837,0:00:00.024911,Não,Não
Adaptada-wlp03,1.146763E+05,1.146812E+05,4.930713E+00,1.148999E+05,1.147428E+05,1.570112E+02,Sim,Sim,0.19%,0.05%,0.14%,0:05:00.333916,0:05:00.037129,0:00:00.296787,Não,Não


In [ ]:
scipDf = pd.DataFrame(resultsData['resultsTask02']['relaxed']['SCIP']).transpose().set_index('instance')
gurobiDf = pd.DataFrame(resultsData['resultsTask02']['relaxed']['gurobi']).transpose().set_index('instance')

mergedDf = pd.merge(scipDf, gurobiDf, on='instance', suffixes=('_SCIP', '_Gurobi'))

mergedDf['primalBoundDifference'] = abs(mergedDf['primalBound_SCIP'] - mergedDf['primalBound_Gurobi'])
mergedDf['dualBoundDifference'] = abs(mergedDf['dualBound_SCIP'] - mergedDf['dualBound_Gurobi'])
mergedDf['timeDifference'] = abs(mergedDf['executionTime_SCIP'] - mergedDf['executionTime_Gurobi'])

mergedDf['dualBound_SCIP'] = mergedDf['dualBound_SCIP'].map(lambda x : '%E' % x)
mergedDf['dualBoundDifference'] = mergedDf['dualBoundDifference'].map(lambda x : '%E' % x)
mergedDf['primalBound_SCIP'] = mergedDf['primalBound_SCIP'].map(lambda x : '%E' % x)
mergedDf['primalBoundDifference'] = mergedDf['primalBoundDifference'].map(lambda x : '%E' % x)
mergedDf['reachedTimeLimit_SCIP'] = mergedDf['reachedTimeLimit_SCIP'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['gap_SCIP'] = mergedDf['gap_SCIP'].map(lambda x : '%.2f%%' % x)
mergedDf['isOptimal_SCIP'] = mergedDf['isOptimal_SCIP'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['executionTime_SCIP'] = mergedDf['executionTime_SCIP'].map(lambda x : str(datetime.timedelta(seconds=x)))
mergedDf['dualBound_Gurobi'] = mergedDf['dualBound_Gurobi'].map(lambda x : '%E' % x)
mergedDf['primalBound_Gurobi'] = mergedDf['primalBound_Gurobi'].map(lambda x : '%E' % x)
mergedDf['reachedTimeLimit_Gurobi'] = mergedDf['reachedTimeLimit_Gurobi'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['timeDifference'] = mergedDf['timeDifference'].map(lambda x : str(datetime.timedelta(seconds=x)))
mergedDf['gap_Gurobi'] = mergedDf['gap_Gurobi'].map(lambda x : '%.2f%%' % (0 if x is None else x))
mergedDf['isOptimal_Gurobi'] = mergedDf['isOptimal_Gurobi'].map(lambda x : 'Sim' if x else 'Não')
mergedDf['executionTime_Gurobi'] = mergedDf['executionTime_Gurobi'].map(lambda x : str(datetime.timedelta(seconds=x)))

mergedDf = mergedDf[['dualBound_SCIP','dualBound_Gurobi','dualBoundDifference','primalBound_SCIP','primalBound_Gurobi','primalBoundDifference','reachedTimeLimit_SCIP','reachedTimeLimit_Gurobi','gap_SCIP','gap_Gurobi','executionTime_SCIP','executionTime_Gurobi','timeDifference','isOptimal_SCIP','isOptimal_Gurobi']]

mergedDf

,dualBound_SCIP,dualBound_Gurobi,dualBoundDifference,primalBound_SCIP,primalBound_Gurobi,primalBoundDifference,reachedTimeLimit_SCIP,reachedTimeLimit_Gurobi,gap_SCIP,gap_Gurobi,executionTime_SCIP,executionTime_Gurobi,timeDifference,isOptimal_SCIP,isOptimal_Gurobi
instance,,,,,,,,,,,,,,,
Adaptada-wlp01,6.900536E+04,6.900536E+04,1.164153E-10,6.900536E+04,6.900536E+04,1.164153E-10,Não,Não,0.00%,0.00%,0:00:18.271101,0:00:00.875817,0:00:17.395284,Sim,Sim
Adaptada-wlp04,1.350690E+05,1.350690E+05,0.000000E+00,1.350690E+05,1.350690E+05,0.000000E+00,Não,Não,0.00%,0.00%,0:04:04.978217,0:00:03.896397,0:04:01.081820,Sim,Sim
Adaptada-wlp05,1.619109E+05,1.619109E+05,8.731149E-11,1.619109E+05,1.619109E+05,8.731149E-11,Não,Não,0.00%,0.00%,0:13:10.657073,0:00:05.849506,0:13:04.807567,Sim,Sim
Adaptada-wlp02,7.584063E+04,7.584063E+04,8.731149E-11,7.584063E+04,7.584063E+04,8.731149E-11,Não,Não,0.00%,0.00%,0:00:29.926196,0:00:01.300919,0:00:28.625277,Sim,Sim
Adaptada-wlp03,1.146337E+05,1.146337E+05,0.000000E+00,1.146337E+05,1.146337E+05,0.000000E+00,Não,Não,0.00%,0.00%,0:01:12.812756,0:00:02.741014,0:01:10.071742,Sim,Sim


In [ ]:
scipDf = pd.DataFrame(resultsData['resultsTask02']['customRelaxed']['SCIP']).transpose().set_index('instance')
gurobiDf = pd.DataFrame(resultsData['resultsTask02']['customRelaxed']['gurobi']).transpose().set_index('instance')

mergedDf = pd.merge(scipDf, gurobiDf, on='instance', suffixes=('_SCIP', '_Gurobi'), how='outer')

mergedDf['primalBoundDifference'] = abs(mergedDf['primalBound_SCIP'] - mergedDf['primalBound_Gurobi'])
mergedDf['dualBoundDifference'] = abs(mergedDf['dualBound_SCIP'] - mergedDf['dualBound_Gurobi'])
mergedDf['timeDifference'] = abs(mergedDf['executionTime_SCIP'] - mergedDf['executionTime_Gurobi'])

mergedDf['dualBound_SCIP'] = mergedDf['dualBound_SCIP'].map(lambda x : 'TLE' if pd.isna(x) else ('%E' % x))
mergedDf['dualBoundDifference'] = mergedDf['dualBoundDifference'].map(lambda x : 'TLE' if pd.isna(x) else ('%E' % x))
mergedDf['primalBound_SCIP'] = mergedDf['primalBound_SCIP'].map(lambda x : 'TLE' if pd.isna(x) else ('%E' % x))
mergedDf['primalBoundDifference'] = mergedDf['primalBoundDifference'].map(lambda x : 'TLE' if pd.isna(x) else ('%E' % x))
mergedDf['reachedTimeLimit_SCIP'] = mergedDf['reachedTimeLimit_SCIP'].map(lambda x : 'TLE' if pd.isna(x) else ('Sim' if x else 'Não'))
mergedDf['gap_SCIP'] = mergedDf['gap_SCIP'].map(lambda x : 'TLE' if pd.isna(x) else ('%.2f%%' % x))
mergedDf['isOptimal_SCIP'] = mergedDf['isOptimal_SCIP'].map(lambda x : 'TLE' if pd.isna(x) else ('Sim' if x else 'Não'))
mergedDf['executionTime_SCIP'] = mergedDf['executionTime_SCIP'].map(lambda x : 'TLE' if pd.isna(x) else (str(datetime.timedelta(seconds=x))))
mergedDf['dualBound_Gurobi'] = mergedDf['dualBound_Gurobi'].map(lambda x : 'TLE' if pd.isna(x) else ('%E' % x))
mergedDf['primalBound_Gurobi'] = mergedDf['primalBound_Gurobi'].map(lambda x : 'TLE' if pd.isna(x) else ('%E' % x))
mergedDf['reachedTimeLimit_Gurobi'] = mergedDf['reachedTimeLimit_Gurobi'].map(lambda x : 'TLE' if pd.isna(x) else ('Sim' if x else 'Não'))
mergedDf['timeDifference'] = mergedDf['timeDifference'].map(lambda x : 'TLE' if pd.isna(x) else (str(datetime.timedelta(seconds=x))))
mergedDf['gap_Gurobi'] = mergedDf['gap_Gurobi'].map(lambda x : 'TLE' if pd.isna(x) else ('%.2f%%' % (0 if x is None else x)))
mergedDf['isOptimal_Gurobi'] = mergedDf['isOptimal_Gurobi'].map(lambda x : 'TLE' if pd.isna(x) else ('Sim' if x else 'Não'))
mergedDf['executionTime_Gurobi'] = mergedDf['executionTime_Gurobi'].map(lambda x : 'TLE' if pd.isna(x) else (str(datetime.timedelta(seconds=x))))

mergedDf = mergedDf[['dualBound_SCIP','dualBound_Gurobi','dualBoundDifference','primalBound_SCIP','primalBound_Gurobi','primalBoundDifference','reachedTimeLimit_SCIP','reachedTimeLimit_Gurobi','gap_SCIP','gap_Gurobi','executionTime_SCIP','executionTime_Gurobi','timeDifference','isOptimal_SCIP','isOptimal_Gurobi']]

mergedDf

,dualBound_SCIP,dualBound_Gurobi,dualBoundDifference,primalBound_SCIP,primalBound_Gurobi,primalBoundDifference,reachedTimeLimit_SCIP,reachedTimeLimit_Gurobi,gap_SCIP,gap_Gurobi,executionTime_SCIP,executionTime_Gurobi,timeDifference,isOptimal_SCIP,isOptimal_Gurobi
instance,,,,,,,,,,,,,,,
Adaptada-wlp01,1.280863E+04,1.280863E+04,4.729372E-11,1.280863E+04,1.280863E+04,4.729372E-11,Não,Não,0.00%,TLE,0:10:10.107975,0:01:14.201814,0:08:55.906161,Sim,Sim
Adaptada-wlp02,1.402980E+04,1.402980E+04,2.473826E-10,1.402980E+04,1.402980E+04,2.473826E-10,Não,Não,0.00%,TLE,0:23:43.641476,0:02:32.190104,0:21:11.451372,Sim,Sim
Adaptada-wlp03,1.812154E+04,1.812154E+04,9.458745E-11,1.812154E+04,1.812154E+04,9.458745E-11,Não,Não,0.00%,TLE,1:57:24.462775,0:10:19.810286,1:47:04.652489,Sim,Sim
Adaptada-wlp04,TLE,2.079708E+04,TLE,TLE,2.079708E+04,TLE,TLE,Não,TLE,TLE,TLE,0:20:00.040341,TLE,TLE,Sim
Adaptada-wlp05,TLE,2.297322E+04,TLE,TLE,2.297322E+04,TLE,TLE,Não,TLE,TLE,TLE,0:36:00.859555,TLE,TLE,Sim
